# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
from azureml.core import Workspace, Experiment

subscription_id = 'f5091c60-1c3c-430f-8d81-d802f6bf2414'
resource_group = 'aml-quickstarts-136663'
workspace_name = 'quick-starts-ws-136663'

ws = Workspace(subscription_id, resource_group, workspace_name)

# choose a name for experiment
experiment_name = 'lab3-experiment'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136663
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-136663


In [5]:
for ds in ws.datasets:
    print(ds)

creditcard
Bank-marketing


In [6]:
dataset = ws.datasets["creditcard"]
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/01-31-2021_121405_UTC/creditcard.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "b243f81e-19d2-49e5-9636-cf000b22e776",
    "name": "creditcard",
    "version": 1,
    "workspace": "Workspace.create(name='quick-starts-ws-136663', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-136663')"
  }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

# the code sninppet is based on official Microsoft documentation, which can be found
# here -> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python
from azureml.core.compute_target import ComputeTargetException

# A name for the cluster
cpu_cluster_name = "lab3-cluster"

# Verification that cluster does not exist yet
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('The cluster already exists')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

The cluster already exists
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name="Class",
    n_cross_validations=5,
    compute_target=cpu_cluster)

In [9]:
# TODO: Submit your experiment
remote_run = exp.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
import joblib

best_automl_run, best_automl_model = remote_run.get_output()

print('Best Run Id: ', best_automl_run.id)

Best Run Id:  AutoML_28ca192b-abc5-4b5c-a8af-d42fe2867002_3


In [13]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_automl_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['1', '0'], 'weights': [0.7777777777777778, 0.2222222222222222]}

1 - maxabsscaler
{'copy': True}

1 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'tree_method': 'auto',
 'verbose': -10,
 'verbosity': 0}



In [14]:
print(best_automl_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                min_child_samples=20,
                                                                                                min_child_weight=0.001,
                                                                                                min_split_gain=0.0,
                                      

In [15]:
#TODO: Save the best model
best_automl_run.register_model(model_name="best_automl_model", model_path="./")

Model(workspace=Workspace.create(name='quick-starts-ws-136663', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-136663'), name=best_automl_model, id=best_automl_model:1, version=1, tags={}, properties={})

In [33]:
os.makedirs('./outputs', exist_ok=True)

joblib.dump(best_automl_model, filename='outputs/automl.joblib')

model_name = best_automl_run.properties['model_name']
model_name

'AutoML28ca192ba3'

In [36]:
# best_automl_run.download_file('outputs/scoring_file_v_0_1.py', 'score.py')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [46]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = best_automl_run.get_environment()

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py', environment=env)

In [52]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'my-custom-env-service'

inference_config = InferenceConfig(entry_script='score.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws, name=service_name,models=[best_automl_model], inference_config=inference_config, deployment_config=aci_config, overwrite=True)
service.wait_for_deployment(show_output=True)

AttributeError: 'Pipeline' object has no attribute 'id'

In [22]:
from azureml.core.model import Model

service = Model.deploy(ws, "payment-fraud-detection", [best_automl_model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

NameError: name 'inference_config' is not defined

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service